In [13]:
import tensorflow as tf  
import numpy as np
import matplotlib.pyplot as plt


In [14]:
mnist = tf.keras.datasets.mnist  
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train / 255.0  
x_test = x_test / 255.0  

In [15]:
l1 = tf.keras.layers.Flatten(input_shape=(28, 28))
l2 = tf.keras.layers.Dense(1000, activation=tf.nn.relu)  
l3 = tf.keras.layers.Dense(1000, activation=tf.nn.relu) 
l4 = tf.keras.layers.Dense(500, activation=tf.nn.relu)
l5 = tf.keras.layers.Dense(200, activation=tf.nn.relu)
op = tf.keras.layers.Dense(10, activation=tf.nn.softmax)

model = tf.keras.models.Sequential() 
model.add(l1)  
model.add(l2)  
model.add(l3)  
model.add(l4) 
model.add(l5)
model.add(op) 

In [16]:
model.compile(optimizer='adam',  
              loss='sparse_categorical_crossentropy',  
              metrics=['accuracy']) 

model.fit(x_train, y_train, epochs=3) 

Epoch 1/3
60000/60000 [==============================] - 85s 1ms/sample - loss: 0.2154 - acc: 0.9360
Epoch 2/3
60000/60000 [==============================] - 84s 1ms/sample - loss: 0.1010 - acc: 0.97101s - loss: 
Epoch 3/3
60000/60000 [==============================] - 93s 2ms/sample - loss: 0.0729 - acc: 0.9798


In [17]:
prunable_weights = [l2.get_weights()[0], l3.get_weights()[0], l4.get_weights()[0], l5.get_weights()[0], op.get_weights()[0]]
prunable_weights

[array([[-0.02191258,  0.01259549,  0.03930573, ..., -0.00736624,
         -0.04897735, -0.00093672],
        [ 0.01131131,  0.01710572, -0.05416338, ...,  0.01650958,
          0.03191972,  0.04387422],
        [ 0.05234427, -0.04108552, -0.02270708, ...,  0.01225932,
         -0.02296691,  0.0392293 ],
        ...,
        [ 0.02865515, -0.01679286, -0.04772637, ..., -0.01545653,
         -0.00290271,  0.04273782],
        [ 0.01562636, -0.04175957, -0.05760852, ...,  0.05294022,
          0.00492226, -0.03615477],
        [-0.02449907, -0.0280659 ,  0.0326672 , ...,  0.00653087,
         -0.02022936, -0.04606252]], dtype=float32),
 array([[ 0.04183542, -0.00999192, -0.00984317, ..., -0.04024448,
          0.01396896, -0.01892357],
        [ 0.05810023,  0.07590514,  0.05791115, ...,  0.02672741,
          0.06498754,  0.05249326],
        [-0.10602294, -0.17064266, -0.12187394, ...,  0.04828503,
          0.06618211, -0.02595068],
        ...,
        [-0.01250048,  0.01431054, -0.0

In [18]:
val_loss, val_acc = model.evaluate(x_test, y_test)
print('Validation loss: ', val_loss)
print('Validation accuracy before pruning: ', val_acc)

10000/10000 [==============================] - 2s 165us/sample - loss: 0.0927 - acc: 0.9749
Validation loss:  0.09271794773282017
Validation accuracy before pruning:  0.9749


In [19]:
prunPerct =[0, 25, 50, 60, 70, 80, 90, 95, 97, 99]
acc = []

In [ ]:
for i in prunPerct:
    pruned_weights = []
    for weight in prunable_weights:
        flattened = np.absolute(np.ndarray.flatten(weight))
        length = len(flattened)
        sort = np.sort(flattened)
        threshold = sort[int((i/100) * length)]
        for row in range(weight.shape[0]):
            for col in range(weight.shape[1]):
                if (abs(weight[row][col]) < threshold):
                    weight[row][col] = 0
    
        pruned_weights.append(weight)
    
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_test = x_test.reshape(x_test.shape[0], 28 * 28) / 255.0
    
    w1 = tf.Variable(initial_value = pruned_weights[0]) 
    w2 = tf.Variable(initial_value = pruned_weights[1])
    w3 = tf.Variable(initial_value = pruned_weights[2])
    w4 = tf.Variable(initial_value = pruned_weights[3])
    
    out_w = tf.Variable(initial_value =pruned_weights[4])
    output = tf.placeholder(tf.int32, [None])
    inputs = tf.placeholder(tf.float32, [None, 784])
    
    l1 = tf.nn.relu(tf.matmul(inputs, w1))
    l2 = tf.nn.relu(tf.matmul(l1, w2))
    l3 = tf.nn.relu(tf.matmul(l2, w3))
    l4 = tf.nn.relu(tf.matmul(l3, w4))
    out = tf.nn.softmax(tf.matmul(l4, out_w))
    predictions = tf.cast(tf.math.argmax(out, axis=1), tf.int32)
    
    acc, acc_percentage = tf.metrics.accuracy(labels=output, predictions=predictions)
    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    accuracy = sess.run(acc_percentage, feed_dict={inputs: x_test, output: y_test})
    print('Accuracy after pruning {}% of weights: {} '.format(i,  accuracy))
    acc.append = accuracy


C:\Users\Sanket\Anaconda3\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Accuracy after pruning 0% of weights: 0.9736999869346619 
Accuracy after pruning 25% of weights: 0.9732999801635742 
Accuracy after pruning 50% of weights: 0.9729999899864197 
Accuracy after pruning 60% of weights: 0.9721999764442444 
Accuracy after pruning 70% of weights: 0.9677000045776367 
Accuracy after pruning 80% of weights: 0.9036999940872192 
Accuracy after pruning 90% of weights: 0.7073000073432922 
Accuracy after pruning 95% of weights: 0.5134000182151794 
Accuracy after pruning 97% of weights: 0.3154999911785126 
